In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
from tqdm import tqdm
import os


from holodeck import plot, detstats, utils
import holodeck.single_sources as sings
from holodeck.constants import YR, MSOL, MPC, GYR, PC
import holodeck as holo
from holodeck.sams import sam

import hasasia.sim as hsim

In [ ]:
RECONSTRUCT_FLAG=False

In [ ]:
SHAPE = None
NREALS = 500
# NREALS = 20
NFREQS = 40
NLOUDEST = 10

NSTEPS = 20

NVARS = 21
# NVARS = 6

NPSRS = 40
NSKIES = 100
# NSKIES = 15

PARAM_NAMES = ['gsmf_phi0', 'gsmf_mchar0_log10',
               'mmb_mamp_log10', 'mmb_scatter_dex', 
               'hard_time', 'hard_gamma_inner']

In [ ]:
def get_data(
        target, nvars=NVARS, nreals=NREALS, nskies=NSKIES, shape=SHAPE, red_gamma = None, red2white=None,
    path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz', load_data_from_file=None,     
):

    if load_data_from_file==None:
        if path == '/Users/emigardiner/GWs/holodeck/output/anatomy_09B':
            load_data_from_file = path+f'/{target}_v{nvars}_r{nreals}_s{nskies}_shape{str(shape)}.npz' 
        else:
            load_data_from_file = path+f'/{target}_v{nvars}_r{nreals}_shape{str(shape)}/data_params.npz' 

    if os.path.exists(load_data_from_file) is False:
        err = f"load data file '{load_data_from_file}' does not exist, you need to construct it."
        raise Exception(err)
    # if os.path.exists(load_dets_from_file) is False:
    #     err = f"load dets file '{load_dets_from_file}' does not exist, you need to construct it."
    #     raise Exception(err)
    file = np.load(load_data_from_file, allow_pickle=True)
    data = file['data']
    params = file['params']
    file.close()

    # file = np.load(load_dets_from_file, allow_pickle=True)
    # dsdat = file['dsdat']
    # file.close()

    return data, params

# Let's just copy Luke's notebook

### function to construct evolution data

In [ ]:
m1_range = [1e7*MSOL, 1e8*MSOL]
m2_range = [1e8*MSOL, 1e9*MSOL]
m3_range = [1e9*MSOL, 1e10*MSOL]
m4_range = [1e10*MSOL, 1e11*MSOL]
mtot_ranges = [m1_range, m2_range, m3_range, m4_range]
mtot_ranges = [mtot_ranges, mtot_ranges]

np.savez('test..npz', mtot_ranges=mtot_ranges)

In [ ]:
m1_range = [1e7*MSOL, 1e8*MSOL]
m2_range = [1e8*MSOL, 1e9*MSOL]
m3_range = [1e9*MSOL, 1e10*MSOL]
m4_range = [1e10*MSOL, 1e11*MSOL]
mtot_ranges = [m1_range, m2_range, m3_range, m4_range]

In [ ]:
def construct_evolution(target_param, params, nsteps, 
    pspace = holo.param_spaces.PS_Uniform_09B(holo.log, nsamples=1, sam_shape=None, seed=None),
):
    """ Construct evolution data.

    saves
    -----
    taus : [nparams][nmasses] array of [nsteps] 1Darrays
        hardening timescales
    target_param_list : [nparams] 
        each of the target's values


    """
    m1_range = [1e7*MSOL, 1e8*MSOL]
    m2_range = [1e8*MSOL, 1e9*MSOL]
    m3_range = [1e9*MSOL, 1e10*MSOL]
    m4_range = [1e10*MSOL, 1e11*MSOL]
    mtot_ranges = [m1_range, m2_range, m3_range, m4_range]
    mrat_range = [0.2, 1.0]
    redz_range = [0, np.inf]


    # Whatever param we're varying
    target_param_list =  []
    # range of binary separations to plot
    sepa = np.logspace(-3, 3, nsteps)[::-1] * PC

    # hcss = []
    # hcbg = []
    taus1 = [] # shape [nvars]
    taus2 = [] # shape [nvars]
    taus3 = [] # shape [nvars]
    taus4 = [] # shape [nvars]

    freq1 = []
    freq2 = []
    freq3 = []
    freq4 = []

    # Iterate over target lifetimes
    for tt in tqdm(range(len(params))):

        # using my parameters from above
        _params = params[tt] # midpoints
        target_param_list.append(_params[target_param])

        sam, hard = pspace.model_for_params(_params)

        # call static_binary_density property to setup variables
        _ = sam.static_binary_density 

        # calculate binary properties at target separations
        _edges, _dnum, _redz_final, _details = sam._dynamic_binary_number_at_sepa_consistent(
            hard, sepa, details=True) # it would be better if I saved these details when I first calculated them!
        
        # select the bins with target binary parameters
        _taus = [] # Shape [nmassses,]
        _freq = []
        for mtot_range in mtot_ranges:
            sel_mtot  = (mtot_range[0] < sam.mtot) & (sam.mtot <= mtot_range[1])
            sel_mrat = (mrat_range[0] < sam.mrat) & (sam.mrat <= mrat_range[1])
            sel_redz = (redz_range[0] < sam.redz) & (sam.redz <= redz_range[1])
            sel = (
                sel_mtot[:, np.newaxis, np.newaxis] *
                sel_mrat[np.newaxis, :, np.newaxis] * 
                sel_redz[np.newaxis, np.newaxis, :]
            )
            _taus.append(_details['tau'][sel].T)
            masses = sam.mtot[sel_mtot]
            print(f"{masses.shape=}, {sepa.shape=}")
            _freq.append(utils.kepler_freq_from_sepa(sam.mtot[sel_mtot,np.newaxis],  
                                                    sepa[np.newaxis,:]))
        print(f"{_taus[0].shape=}")   
            
        taus1.append(_taus[0])
        taus2.append(_taus[1])
        taus3.append(_taus[2])
        taus4.append(_taus[3])
        
        freq1.append(_freq[0])
        freq2.append(_freq[1])
        freq3.append(_freq[2])
        freq4.append(_freq[3])

    # save results
    fileloc = '/Users/emigardiner/GWs/holodeck/ecg-notebooks/parameter_investigation/anatomy_uniform09B/'
    filename = fileloc+f'evol_{target_param}.npz' 
    print(f"{filename=}")
    np.savez('test.npz', test=taus1)
    np.savez('test.npz', test=target_param_list)
    np.savez('test.npz', test=nsteps)
    np.savez('test.npz', test=sepa)
    np.savez('test.npz', taus=mtot_ranges)

    np.savez(filename, target_param_list=target_param_list,
            nsteps=nsteps, sepa=sepa,
            mtot_ranges = mtot_ranges,
            taus1=taus1, 
            taus2=taus2, 
            taus3=taus3, 
            taus4=taus4, 
            freq1=freq1, # M, nsteps
            freq2=freq2, 
            freq3=freq3, 
            freq4=freq4,)

    rv = dict(
        taus1=taus1, taus2=taus2, taus3=taus3, taus4=taus4,
        freq1=freq1, freq2=freq2, freq3=freq3, freq4=freq4,
        target_list=target_param_list, nsteps=nsteps, sepa=sepa,
        mtot_ranges=mtot_ranges)
    print(rv)
    return rv

# hard_gamma_inner

In [ ]:
target='hard_gamma_inner'
 
data, params = get_data(target)
data = [data[0], data[10], data[20]]
params = [params[0], params[10], params[20]]
fobs_gw_cents = data[0]['fobs_cents']
_, fobs_gw_edges = holo.utils.pta_freqs()

In [ ]:
RECONSTRUCT_FLAG=False

In [ ]:
if RECONSTRUCT_FLAG:
    rv_hard_gamma_inner = construct_evolution(target_param=target, params=params, nsteps=NSTEPS)
else:
    fileloc = '/Users/emigardiner/GWs/holodeck/ecg-notebooks/parameter_investigation/anatomy_uniform09B/'
    filename = fileloc+f'evol_{target}_rv.npz'
    file = np.load(filename, allow_pickle=True)
    rv_hard_gamma_inner = file['rv']
    file.close()

In [ ]:
print(rv_hard_gamma_inner.keys())

In [ ]:
rv = rv_hard_gamma_inner

# Define colors and linestyles
colors = [
    ['#80ccff', '#065de0', '#0000b3',], # m lowest
    ['#5eed5e', '#00b32d', '#006600'], # m2
    ['#ffcc00', '#aa8109', '#805500'], # m3
    ['#ff8080', '#e61919', '#730d0d']] # m4
linestyles = [':', '--', '-']

# Make figure
fig, axs = plot.figax_double(ncols=2, height=4)
xlabel1 = "Separation (pc)"
xlabel2 = "Frequency ($\mathrm{yr}^{-1}$)"
fig.suptitle(f"{plot.PARAM_KEYS['hard_time']}={params[0]['hard_time']} GYr")


# Set labels and ticks
axs[0].set_xlabel(xlabel1)
axs[1].set_xlabel(xlabel2)
axs[0].set_ylabel(plot.LABEL_HARDENING_TIME)
axs[1].set_yticklabels([])

x1 = rv['sepa'] # S
yys = [rv['taus1'], rv['taus2'], rv['taus3'], rv['taus4']] # [4masses, 3vars, M, Q, Z]

mtots = 10**np.mean(np.log10(mtot_ranges), axis=-1)

mass_handles = []
mass_labels = []

target_handles = []
target_labels = []

for vv, targ in enumerate(rv['target_list']):
    for mm, mtot in enumerate(mtots):
        x2 = utils.kepler_freq_from_sepa(np.ones(len(x1))*mtot, x1) # frequencies
        hh, gg = plot.draw_med_conf_color(axs[0], x1/PC, yys[mm][vv]/GYR, 
                                    color=colors[mm][vv], linestyle=linestyles[vv])
        plot.draw_med_conf_color(axs[1], x2*YR, yys[mm][vv]/GYR, 
                                    color=colors[mm][vv], linestyle=linestyles[vv])
        if vv==1: 
            mass_handles.append(hh)
            mass_labels.append(f"~{mtot/MSOL:.1e} $M_\odot$")
        if mm==0:
            target_handles.append(hh)
            target_labels.append(targ)

fig.subplots_adjust(wspace=0)
axs[1].axvline(1/20, linestyle='--', color='k', alpha=0.5)
axs[1].axvline(1, linestyle='--', color='k', alpha=0.5)

xlim0 = axs[0].get_xlim()
axs[0].set_xlim(xlim0[1], xlim0[0])

axs[0].legend(handles=mass_handles, labels=mass_labels, fontsize=12, title='Mass')
axs[1].legend(handles=target_handles, labels=target_labels, fontsize=12, title=plot.PARAM_KEYS[target])



In [ ]:
rv = rv_hard_gamma_inner

# Define colors and linestyles
colors = [
    ['#80ccff', '#065de0', '#0000b3',], # m lowest
    ['#5eed5e', '#00b32d', '#006600'], # m2
    ['#ffcc00', '#aa8109', '#805500'], # m3
    ['#ff8080', '#e61919', '#730d0d']] # m4
linestyles = [':', '--', '-']

# Make figure
fig, axs = plot.figax_double(ncols=2, height=4)
xlabel1 = "Separation (pc)"
xlabel2 = "Frequency ($\mathrm{yr}^{-1}$)"
fig.suptitle(f"{plot.PARAM_KEYS['hard_time']}={params[0]['hard_time']} GYr")


# Set labels and ticks
axs[0].set_xlabel(xlabel1)
axs[1].set_xlabel(xlabel2)
axs[0].set_ylabel(plot.LABEL_HARDENING_TIME)
axs[1].set_yticklabels([])

x1 = rv['sepa'] # S
yys = [rv['taus1'], rv['taus2'], rv['taus3'], rv['taus4']] # [4masses, 3vars, M, Q, Z]

mtots = 10**np.mean(np.log10(mtot_ranges), axis=-1)

mass_handles = []
mass_labels = []

target_handles = []
target_labels = []

for vv, targ in enumerate(rv['target_list']):
    for mm, mtot in enumerate(mtots):
        x2 = utils.kepler_freq_from_sepa(np.ones(len(x1))*mtot, x1) # frequencies
        hh, gg = plot.draw_med_conf_color(axs[0], x1/PC, yys[mm][vv]/GYR, 
                                    color=colors[mm][vv], linestyle=linestyles[vv], fracs=[0.5,])
        plot.draw_med_conf_color(axs[1], x2*YR, yys[mm][vv]/GYR, 
                                    color=colors[mm][vv], linestyle=linestyles[vv], fracs=[0.5,])
        if vv==1: 
            mass_handles.append(hh)
            mass_labels.append(f"~{mtot/MSOL:.1e} $M_\odot$")
        if mm==0:
            target_handles.append(hh)
            target_labels.append(targ)

fig.subplots_adjust(wspace=0)
# axs[1].set_xlim(1/30, 3)
xlim0 = axs[0].get_xlim()
axs[0].set_xlim(xlim0[1], xlim0[0])
axs[1].axvline(1/16.3, linestyle='-', color='k', alpha=0.5)
axs[1].axvline(1, linestyle='-', color='k', alpha=0.5)

axs[0].legend(handles=mass_handles, labels=mass_labels, fontsize=12, title='Mass', loc='lower left')
axs[1].legend(handles=target_handles, labels=target_labels, fontsize=12, title=plot.PARAM_KEYS[target], loc='lower left')



In [ ]:
rv = rv_hard_gamma_inner

# Define colors and linestyles
colors = [
    ['#80ccff', '#065de0', '#0000b3',], # m lowest
    ['#5eed5e', '#00b32d', '#006600'], # m2
    ['#ffcc00', '#aa8109', '#805500'], # m3
    ['#ff8080', '#e61919', '#730d0d']] # m4
linestyles = [':', '--', '-']

# Make figure
fig, ax = plot.figax_single(ncols=1, )
# xlabel1 = "Separation (pc)"
xlabel2 = "Frequency ($\mathrm{yr}^{-1}$)"
# fig.suptitle(f"{plot.PARAM_KEYS['hard_time']}={params[0]['hard_time']} GYr")


# Set labels and ticks
# axs[0].set_xlabel(xlabel1)
ax.set_xlabel(xlabel2)
ax.set_ylabel(plot.LABEL_HARDENING_TIME)
# ax.set_yticklabels([])

x1 = rv['sepa'] # S
yys = [rv['taus1'], rv['taus2'], rv['taus3'], rv['taus4']] # [4masses, 3vars, M, Q, Z]

mtots = 10**np.mean(np.log10(mtot_ranges), axis=-1)

mass_handles = []
mass_labels = []

target_handles = []
target_labels = []

for vv, targ in enumerate(rv['target_list']):
    for mm, mtot in enumerate(mtots):
        x2 = utils.kepler_freq_from_sepa(np.ones(len(x1))*mtot, x1) # frequencies
        # hh, gg = plot.draw_med_conf_color(axs[0], x1/PC, yys[mm][vv]/GYR, 
        #                             color=colors[mm][vv], linestyle=linestyles[vv], fracs=[0.5,])
        hh, gg = plot.draw_med_conf_color(ax, x2*YR, yys[mm][vv]/GYR, 
                                    color=colors[mm][vv], linestyle=linestyles[vv], fracs=[0.5,])
        if vv==1: 
            mass_handles.append(hh)
            mass_labels.append(f"~{mtot/MSOL:.1e} $M_\odot$")
        if mm==0:
            target_handles.append(hh)
            target_labels.append(targ)

fig.subplots_adjust(wspace=0)
# axs[1].set_xlim(1/30, 3)
# xlim0 = ax.get_xlim()
# axs[0].set_xlim(xlim0[1], xlim0[0])
ax.axvline(data[0]['fobs_cents'][0]*YR, linestyle='-', color='k', alpha=0.5)
ax.axvline(data[0]['fobs_cents'][-1]*YR, linestyle='-', color='k', alpha=0.5)

leg1 = ax.legend(handles=mass_handles, labels=mass_labels, fontsize=12, title='Mass', loc='lower left')
leg2 = ax.legend(handles=target_handles, labels=target_labels, fontsize=12, title=plot.PARAM_KEYS[target], 
                 bbox_to_anchor=(0.36, 0.00), loc='lower left')

ax.add_artist(leg1)

savename = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots'+'/evolution.png'
fig.savefig(savename, dpi=100, bbox_inches='tight')

Observations
* flat gamma_inner doesn't change massive sources in PTA band much, 
* 

## Use short hardening time

In [ ]:
target='hard_gamma_inner'
 
data, params = get_data(target, 
    load_data_from_file='/Users/emigardiner/GWs/holodeck/output/anatomy_redz/hard_gamma_inner_v5_r100_shapeNone/data_params.npz')
data = [data[0], data[2], data[4]]
params = [params[0], params[2], params[4]]
fobs_gw_cents = data[0]['fobs_cents']
_, fobs_gw_edges = holo.utils.pta_freqs()

In [ ]:
if RECONSTRUCT_FLAG:
    rv_hard_gamma_inner2 = construct_evolution(target_param=target, params=params, nsteps=NSTEPS)
else:
    fileloc = '/Users/emigardiner/GWs/holodeck/ecg-notebooks/parameter_investigation/anatomy_uniform09B/'
    filename = fileloc+f'evol_{target}_rv.npz'
    file = np.load(filename, allow_pickle=True)
    rv_hard_gamma_inner2 = file['rv']
    file.close()

In [ ]:
rv = rv_hard_gamma_inner2

# Define colors and linestyles
colors = [
    ['#80ccff', '#065de0', '#0000b3'], # m lowest
    ['#5eed5e', '#00b32d', '#006600'], # m2
    ['#ffcc00', '#aa8109', '#805500'], # m3
    ['#ff8080', '#e61919', '#730d0d']] # m4
linestyles = [':', '--', '-']

# Make figure
fig, axs = plot.figax_double(ncols=2, height=4)
xlabel1 = "Separation (pc)"
xlabel2 = "Frequency ($\mathrm{yr}^{-1}$)"
fig.suptitle(f"{plot.PARAM_KEYS['hard_time']}={params[0]['hard_time']} GYr")

# Set labels and ticks
axs[0].set_xlabel(xlabel1)
axs[1].set_xlabel(xlabel2)
axs[0].set_ylabel(plot.LABEL_HARDENING_TIME)
axs[1].set_yticklabels([])

x1 = rv['sepa'] # S
yys = [rv['taus1'], rv['taus2'], rv['taus3'], rv['taus4']] # [4masses, 3vars, M, Q, Z]

mtots = 10**np.mean(np.log10(mtot_ranges), axis=-1)

mass_handles = []
mass_labels = []

target_handles = []
target_labels = []

for vv, targ in enumerate(rv['target_list']):
    for mm, mtot in enumerate(mtots):
        x2 = utils.kepler_freq_from_sepa(np.ones(len(x1))*mtot, x1) # frequencies
        hh, gg = plot.draw_med_conf_color(axs[0], x1/PC, yys[mm][vv]/GYR, 
                                    color=colors[mm][vv], linestyle=linestyles[vv], fracs=[0.5,])
        plot.draw_med_conf_color(axs[1], x2*YR, yys[mm][vv]/GYR, 
                                    color=colors[mm][vv], linestyle=linestyles[vv], fracs=[0.5,])
        if vv==1: 
            mass_handles.append(hh)
            mass_labels.append(f"~{mtot/MSOL:.1e} $M_\odot$")
        if mm==0:
            target_handles.append(hh)
            target_labels.append(targ)

fig.subplots_adjust(wspace=0)
# axs[1].set_xlim(1/30, 3)
xlim0 = axs[0].get_xlim()
axs[0].set_xlim(xlim0[1], xlim0[0])
axs[1].axvline(1/16.3, linestyle='-', color='k', alpha=0.5)
axs[1].axvline(1, linestyle='-', color='k', alpha=0.5)

axs[0].legend(handles=mass_handles, labels=mass_labels, fontsize=12, title='Mass', loc='lower left')
axs[1].legend(handles=target_handles, labels=target_labels, fontsize=12, title=plot.PARAM_KEYS[target], loc='lower left')

print(f"{x1.shape=}, {x2s[0][0].shape=}, {yys[0][0].shape=}")

    np.savez(filename, taus=taus, target_param_list=target_param_list,
            nsteps=nsteps, sepa=sepa,
            mtot_ranges = mtot_ranges)

In [ ]:

    
np.savez(filename, rv=rv)

file = np.load(filename, allow_pickle=True)
rv = file['rv']

In [ ]:
def load_evolution(target):
    fileloc = '/Users/emigardiner/GWs/holodeck/ecg-notebooks/parameter_investigation/anatomy_uniform09B/'
    filename = fileloc+f'evol_{target}.npz'
    file = np.load(filename, allow_pickle=True)
    taus1=file['taus1']
    taus2=file['taus2']
    taus3=file['taus3']
    taus4=file['taus4']
    freq1=file['freq1']
    freq2=file['freq2']
    freq3=file['freq3']
    freq4=file['freq4']
    target_list = file['target_param_list']
    nsteps = file['nsteps']
    sepa = file['sepa']
    mtot_ranges = file['mtot_ranges']
    file.close()
    rv = dict(
        taus1=taus1, taus2=taus2, taus3=taus3, taus4=taus4,
        freq1=freq1, freq2=freq2, freq3=freq3, freq4=freq4,
        target_list=target_list, nsteps=nsteps, sepa=sepa)

In [ ]:
file = np.load('/Users/emigardiner/GWs/holodeck/ecg-notebooks/parameter_investigation/anatomy_uniform09B/evol_hard_gamma_inner.npz',
            allow_pickle=True)

In [ ]:
rv = load_evolution('hard_gamma_inner')

In [ ]:
target_param='hard_time'
fileloc = '/Users/emigardiner/GWs/holodeck/ecg-notebooks/parameter_investigation/anatomy_uniform09A_fullshape'
filename = fileloc+'/evol_%s_%dsteps.npz' % (target_param, nsteps)
file = np.load(filename)
taus=file['taus']
taus_high=file['taus_high'] 
target_param_list=file['target_param_list']
hcss=file['hcss']
hcbg=file['hcbg']
nsteps=file['nsteps']
sepa=file['sepa']
mtot_range=file['mtot_range'] 
mtot_hirng=file['mtot_hirng']
mrat_range=file['mrat_range']
redz_range=file['redz_range']
file.close()

## load gamma_inner results

In [ ]:
fileloc = '/Users/emigardiner/GWs/holodeck/ecg-notebooks/parameter_investigation/anatomy_uniform09B/'
filename = fileloc+'evol_%s_%dsteps.npz' % (target, nsteps)
file = np.load(filename)
taus=file['taus']
taus_high=file['taus_high'] 
target_param_list=file['target_param_list']
hcss=file['hcss']
hcbg=file['hcbg']
nsteps=file['nsteps']
sepa=file['sepa']
mtot_range=file['mtot_range'] 
mtot_hirng=file['mtot_hirng']
mrat_range=file['mrat_range']
redz_range=file['redz_range']
file.close()

## plot results

In [ ]:
def plot_current():
    fig, ax = plot.figax_single(ncols=2, sharey=True)

    xx = sepa/PC
    x1_m1 = utils.kepler_freq_from_sepa(mass, sepa)
    YR_LABEL_PAD = -4
    colors = ['tab:green', 'tab:blue', 'tab:orange']

    # ------------------------   Ax Row 0   ----------------------------

    ax1.sharex(ax)
    ax1.sharey(ax)

    for axis in [ax, ax1]:
        axis.set(xlabel=plot.LABEL_SEPARATION_PC, ylabel=plot.LABEL_HARDENING_TIME, xscale='log', yscale='log')
    axis.invert_xaxis()

        # axis.axhline(times_list[1], color='k', alpha=0.65)
        # axis.axhline(times_list[0], color='k', ls='--', alpha=0.25)


    labels = []
    handles = []
    for ii, tau in enumerate(taus):
        print(ii)
        yy = tau / GYR
        hh = plot.draw_med_conf_color(ax, xx, yy, fracs=[0.5], filter=True, color=colors[ii])
        # colors.append(hh[0].get_color())
        handles.append(hh[0])
        labels.append(f"${target_param_list[ii]:.1f}$")

        y1 = taus_high[ii]/GYR   
        plot.draw_med_conf_color(ax1, xx, y1, fracs=[0.5], filter=True, color=colors[ii])



    leg = ax.legend(handles, labels, loc='lower left', 
                    ncol=len(handles), title=target_param, title_fontsize=14)

    # ----------------------------- Ax Row 1 --------------------------------

    ax = axs[1,0]
    ax1 = axs[1,1]

    ax1.sharex(ax)
    ax1.sharey(ax)

    for axis in [ax, ax1]:
        axis.set(xlabel=plot.LABEL_GW_FREQUENCY_NHZ, ylabel=plot.LABEL_CHARACTERISTIC_STRAIN, xscale='log', yscale='log')

    xx = fobs_gw_cents*1e9 # nHz

    labels=[]
    handles=[]
    # colors = []

    for ii, yy in enumerate(hcbg):
        # yy = np.median(yy, axis=-1)
        hh = plot.draw_med_conf_color(ax, xx, yy, fracs=[0.5], filter=False, color=colors[ii])
        # colors.append(hh[0].get_color())
        ss = hcss[ii]
        for rr in range(len(ss[0])):
            ax.scatter(xx, ss[:,rr,0], color = colors[ii], alpha=0.5, s=5) # only single include loudest of each realization

    return fig

In [ ]:
fig = plot_current()